In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.python.framework.ops import disable_eager_execution

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split
from sklearn import metrics

import plotly
import plotly.graph_objs as go
import plotly.express as px

K = tf.keras.backend

In [ ]:
disable_eager_execution()
tf.executing_eagerly()

In [ ]:
np.random.seed(seed=13)

EPOCH = 500

In [ ]:
dataset = pd.read_csv('Weather.csv')
dataset.shape

In [ ]:
dataset.describe()

In [ ]:
dataset.plot(x='MinTemp', y='MaxTemp', style='o')  
plt.title('MinTemp vs MaxTemp')  
plt.xlabel('MinTemp')  
plt.ylabel('MaxTemp')
plt.savefig('min_max_temp.jpeg', dpi=300)

In [ ]:
plt.figure(figsize=(15,10))
plt.tight_layout()
seabornInstance.distplot(dataset['MaxTemp'])
plt.savefig('dis_max_temp.jpeg', dpi=300)

In [ ]:
x = dataset['MinTemp'].values.reshape(-1,1)
y = dataset['MaxTemp'].values.reshape(-1,1)

x.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle= True)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
W = tf.Variable(tf.random.uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random.uniform([1], -1.0, 1.0))

y = W * x_train + b

In [ ]:
loss = tf.reduce_mean(tf.square(y - y_train))

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.0001)

train = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()
sess.run(init)

In [ ]:
history = []
wb = []

for step in range(EPOCH):
    sess.run(train)
    history.append(sess.run(loss))
    print(step, sess.run(W), sess.run(b), sess.run(loss))
    wb.append([sess.run(W)[0], sess.run(b)[0], sess.run(loss)])

In [ ]:
M = sess.run(W)
C = sess.run(b)

In [ ]:
def predict(X, M, C):
    y = M*X+C
    return y[0]

In [ ]:
df = pd.DataFrame(history, columns=['loss'])

In [ ]:
h1 = go.Scatter(y=df['loss'], 
                    mode="lines",
                    line=dict(
                    	width=2,
                    	color='blue'),
                    name="loss")

data = [h1]

layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='Epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
y_pred = [predict(i, M, C) for i in x_test]

y_test.shape

In [ ]:
y_test = y_test.reshape(-1)
y_test.shape

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.head(10)

In [ ]:
df1 = df.head(10)
df1.plot(kind='bar',figsize=(15,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.savefig('actual-predict.jpeg', dpi=300)

In [ ]:
M = [i[0] for i in wb]
L = [i[2] for i in wb]
C = [i[1] for i in wb]

In [ ]:
y_pred = [predict(i, M[0], C[0]) for i in x_test]

plt.scatter(x_test, y_test,  color='gray')
plt.plot(x_test, y_pred, color='red', linewidth=2)
plt.savefig('min_max_temp1.jpeg', dpi=300)

In [ ]:
y_pred = [predict(i, M[4], C[4]) for i in x_test]

plt.scatter(x_test, y_test,  color='gray')
plt.plot(x_test, y_pred, color='red', linewidth=2)
plt.savefig('min_max_temp5.jpeg', dpi=300)

In [ ]:
y_pred = [predict(i, M[9], C[9]) for i in x_test]

plt.scatter(x_test, y_test,  color='gray')
plt.plot(x_test, y_pred, color='red', linewidth=2)
plt.savefig('min_max_temp10.jpeg', dpi=300)

In [ ]:
y_pred = [predict(i, M[499], C[499]) for i in x_test]

plt.scatter(x_test, y_test,  color='gray')
plt.plot(x_test, y_pred, color='red', linewidth=2)
plt.savefig('min_max_temp500.jpeg', dpi=300)

In [ ]:
plt.scatter(M, L,  color='gray')
plt.savefig('weight.jpeg', dpi=300)

In [ ]:
plt.scatter(C, L, color='gray')
plt.savefig('bias.jpeg', dpi=300)

In [ ]:
import plotly.express as px

df = pd.DataFrame({'W' : M, 'Bias' : C, 'Loss' : L})
fig = px.scatter_3d(df, x='W', y='Bias', z='Loss')
fig.show()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

# **Stochastic Gradient Descent Method**
**Linear Regression with Keras**

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1, input_dim=1, kernel_initializer='random_uniform', activation='linear'))
model.summary()

In [ ]:
SGD = tf.keras.optimizers.SGD
sgd = SGD(learning_rate=0.0001)

model.compile(loss='mse', optimizer=sgd, metrics=['mae', 'mse', rmse])

In [ ]:
history = model.fit(x_train, y_train, epochs=EPOCH, batch_size=64,  verbose=1, validation_split=0.2, shuffle=True)

In [ ]:
h1 = go.Scatter(y=history.history['loss'], 
                    mode="lines",
                    line=dict(
                    	width=2,
                    	color='blue'),
                    name="loss")

h2 = go.Scatter(y=history.history['val_loss'], 
                    mode="lines",
                    line=dict(
                    	width=2,
                    	color='green'),
                    name="val_loss")
                    
data = [h1, h2]

layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred = y_pred.flatten()

df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.head(10)

In [ ]:
df1 = df.head(10)
df1.plot(kind='bar',figsize=(15,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.savefig('actual-predict2.jpeg', dpi=300)

In [ ]:
plt.scatter(x_test, y_test,  color='gray')
plt.plot(x_test, y_pred, color='red', linewidth=2)
plt.savefig('keras_500_model.jpeg', dpi=300)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
